In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# open train set csv as dataframe
data = pd.read_csv('/content/drive/MyDrive/ESA_DATA/21_months.train.csv')
print(data.shape)
data.head()
# non-target channels: 1-11, 53-56, 67-69, out of 76 total channels (the others are target channels)
# telecommands are binary (0 or 1)

In [ ]:
# count number channel containing at least 1 anomaly using the binary columns "is_anomaly_channel{n}"
anomalies = data.filter(like='is_anomaly_channel')
mask = anomalies.sum() != 0
len(anomalies.columns[mask])

In [ ]:
mask_nominal = anomalies.sum() == 0
len(anomalies.columns[mask_nominal])

In [ ]:
plt.plot(data['channel_12'])
# plot a vertical line where columns "is_anomaly_channel_12" is greater than 0
plt.vlines(data.index[data['is_anomaly_channel_12'] != 0], ymin=0.16, ymax=0.3, color='red')

In [ ]:
data['is_anomaly_channel_12'].unique() # nominal, anomaly, rare_event, communication_gap

array([0, 1, 2, 3])

In [ ]:
#extract consecutive points from "train" such that there are no anomalies in any of the channels, therefore splitting the data into several 2 subseries (before the anomaly occurs and after up to the next anomaly) for each anomaly
def extract_clean_consecutive_timepoints(train_data):
    # find the indices of the anomalies
    anomaly_indices = train_data.index[train_data.filter(like='is_anomaly_channel').sum(axis=1) != 0]
    # create a list to store the subseries
    subseries = []
    # initialize the start index of the subseries
    start_idx = -1
    # iterate over the anomaly indices
    for idx in anomaly_indices:
        # extract the subseries from the start index to the anomaly index
        if idx - start_idx == 1:
            start_idx = idx
            continue
        subseries.append(train_data.iloc[start_idx + 1:idx].filter(regex='^channel'))
        # update the start index to the next index
        start_idx = idx
    # extract the last subseries from the last anomaly index to the end of the data
    subseries.append(train_data.iloc[start_idx+1:].filter(regex='^channel'))
    return subseries


In [ ]:
def segment_time_series(dataframes, window_length=256, overlap=0.5):
    """
    Segments a list of dataframes representing time series into windows of fixed length with overlap.
    Discards dataframes shorter than the window length and trims any remainder for imperfect divisions.

    Parameters:
    - dataframes: List of pandas DataFrame objects to segment.
    - window_length: The length of each segment (default: 256).
    - overlap: Fraction of overlap between consecutive windows (default: 0.5).

    Returns:
    - List of pandas DataFrame objects, each with the specified window length.
    """
    segmented_dfs = []
    step = int(window_length * (1 - overlap))  # Step size based on overlap

    for df in dataframes:
        length = len(df)
        if length < window_length:
            continue  # Discard short time series
        num_windows = (length - window_length) // step + 1  # Calculate number of full windows (similar to convolution shape calculation)
        for i in range(num_windows):
            start = i * step
            end = start + window_length
            segmented_dfs.append(df.iloc[start:end])

    return segmented_dfs

In [ ]:
# divide the dataframe timeseries into training and test set (two halves)
def train_test_division(df, p=0.54):
    index = int(len(df)*p)
    training_df = df.iloc[:index]
    test_df = df.iloc[index:]
    training_series = extract_clean_consecutive_timepoints(training_df)
    training_series = segment_time_series(training_series)
    # for training keep only columns starting with "channel_", for test keep those and also "is_anomaly_channel"
    test_df = test_df.filter(regex='^channel|^is_anomaly_channel')
    test_series = segment_time_series([test_df])
    # divide the whole dataframe into non_anomalous subseries for the training set, keep
    return training_series, test_series

training_series, test_series = train_test_division(data)
print(len(training_series))
print(len(test_series))

In [ ]:
test_series[42]

,channel_1,channel_10,channel_11,channel_12,channel_13,channel_14,channel_15,channel_16,channel_17,channel_18,...,is_anomaly_channel_7,is_anomaly_channel_70,is_anomaly_channel_71,is_anomaly_channel_72,is_anomaly_channel_73,is_anomaly_channel_74,is_anomaly_channel_75,is_anomaly_channel_76,is_anomaly_channel_8,is_anomaly_channel_9
999149,0.13791,0.0,0.0,0.248549,0.301578,0.267029,0.090177,0.680347,0.275598,0.284645,...,0,0,0,0,0,0,0,0,0,0
999150,0.13791,0.0,0.0,0.248549,0.301578,0.267029,0.090177,0.680347,0.275598,0.284645,...,0,0,0,0,0,0,0,0,0,0
999151,0.13791,0.0,0.0,0.248549,0.301578,0.267029,0.088891,0.678508,0.275598,0.284645,...,0,0,0,0,0,0,0,0,0,0
999152,0.13791,0.0,0.0,0.248549,0.301578,0.267029,0.088891,0.678508,0.275598,0.284645,...,0,0,0,0,0,0,0,0,0,0
999153,0.13791,0.0,0.0,0.248549,0.301578,0.267029,0.088891,0.678508,0.275598,0.284645,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999400,0.13791,0.0,0.0,0.243869,0.298458,0.267029,0.088891,0.687702,0.277106,0.290674,...,0,0,0,0,0,0,0,0,0,0
999401,0.13791,0.0,0.0,0.243869,0.298458,0.267029,0.088891,0.687702,0.277106,0.290674,...,0,0,0,0,0,0,0,0,0,0
999402,0.13791,0.0,0.0,0.243869,0.298458,0.267029,0.088891,0.687702,0.277106,0.290674,...,0,0,0,0,0,0,0,0,0,0
999403,0.13791,0.0,0.0,0.243869,0.298458,0.267029,0.088891,0.678508,0.277106,0.292182,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in range(len(train_series)):
    if train_series[i].filter(like='is_anomaly_channel').sum().sum() != 0:
        print('ANOMALY FOUND something went wrong')
print("END")
print(len(train_series))

END
28


In [ ]:
from torch.utils.data import Dataset, DataLoader

class AnomalyDetectionDataset(Dataset):
    def __init__(self, dataframes, non_target_channels):
        """
        Args:
            dataframes (list of pd.DataFrame): List of dataframes, each representing a multivariate time series.
            non_target_channels (list of int): List of channel numbers to be separated as non-target channels.
        """
        self.dataframes = dataframes
        self.non_target_channels = [f"channel_{n}" for n in non_target_channels]

    def __len__(self):
        return len(self.dataframes)

    def __getitem__(self, idx):
        # Get the dataframe at the specified index
        df = self.dataframes[idx]

        # Separate the non-target channels
        non_target_df = df[self.non_target_channels]

        # Remaining channels are target channels
        target_df = df.drop(columns=self.non_target_channels)

        # Convert dataframes to PyTorch tensors
        target_tensor = torch.tensor(target_df.values, dtype=torch.float32)
        non_target_tensor = torch.tensor(non_target_df.values, dtype=torch.float32)

        return target_tensor, non_target_tensor

In [ ]:
train_dataset = AnomalyDetectionDataset(dataframes[:8], non_target_channels)
test_dataset = AnomalyDetectionDataset(dataframes[8:], non_target_channels)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:
import pytorch_lightning as pl
from torch import nn

class VRAE(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_layers, learning_rate=1e-3):
        super(VRAE, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        self.num_layers = num_layers
        self.learning_rate = learning_rate

        # Encoder (Bi-LSTM)
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.hidden_to_latent = nn.Linear(hidden_dim * 2, latent_dim)
        self.latent_to_mu = nn.Linear(latent_dim, latent_dim)
        self.latent_to_logvar = nn.Linear(latent_dim, latent_dim)

        # Decoder (LSTM)
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim)
        self.decoder_cell = nn.LSTMCell(input_dim, hidden_dim)
        self.hidden_to_output = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        _, (hidden, _) = self.encoder(x)
        hidden = hidden.view(hidden.size(1), -1)  # Concatenate bidirectional outputs -> (batch, hidden_dim*2)
        latent = self.hidden_to_latent(hidden)
        mu = self.latent_to_mu(latent)
        logvar = self.latent_to_logvar(latent)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, seq_len):
        batch_size = z.size(0)

        # Initialize the hidden and cell states for the LSTMCell using the latent vector z
        hidden = self.latent_to_hidden(z)  # Shape: [batch_size, hidden_dim]
        cell = torch.zeros(batch_size, self.hidden_dim, device=z.device)

        # Initialize the first input to the decoder as zeros
        decoder_input = torch.zeros(batch_size, self.input_dim, device=z.device)  # Shape: [batch_size, input_dim]
        outputs = []

        # Autoregressively decode the sequence
        for _ in range(seq_len):
            hidden, cell = self.decoder_cell(decoder_input, (hidden, cell))
            decoder_input = self.hidden_to_output(hidden)  # Use output as next input directly
            outputs.append(decoder_input.unsqueeze(1))

        return torch.cat(outputs, dim=1)  # Shape: [batch_size, seq_len, input_dim]

    def forward(self, x):
        seq_len = x.size(1)
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_reconstructed = self.decode(z, seq_len)
        return x_reconstructed, mu, logvar

    def training_step(self, batch, batch_idx):
        target_batch, _ = batch
        x_reconstructed, mu, logvar = self(target_batch)
        recon_loss = nn.MSELoss()(x_reconstructed, target_batch)
        kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / target_batch.size(0)
        loss = recon_loss + kld_loss
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)
